In [1]:
from lsc_repeats import LSCRepeatSample
from libra_toolbox.tritium.model import ureg, Model, quantity_to_activity
import numpy as np
import json
from libra_toolbox.tritium.lsc_measurements import (
    LIBRARun,
    LSCFileReader,
    GasStream,
    LSCSample,
    LIBRASample,
)
from datetime import datetime

all_file_readers = []
all_quench = []

In [20]:
def create_repeat_sample(label: str, filename: str) -> LSCRepeatSample:
    """
    Create a LSCRepeatSample from a LSC file with background substracted.

    Args:
        label: the label of the sample in the LSC file
        filename: the filename of the LSC file

    Returns:
        the LSCRepeatSample object
    """
    # check if a LSCFileReader has been created for this filename
    found = False
    for file_reader in all_file_readers:
        if file_reader.filename == filename:
            found = True
            break

    # if not, create it and add it to the list of LSCFileReaders
    if not found:
        file_reader = LSCFileReader(filename, labels_column="SMPL_ID")

    file_reader.read_file()

    # create the sample
    sample = LSCRepeatSample.from_file(file_reader, label)

   # try to find the background sample from the file
    background_labels = ["1L-BL-1", "1L-BL-2", "1L-BL-3"]
    background_sample = None

    for background_label in background_labels:
        try:
            background_sample = LSCRepeatSample.from_file(file_reader, background_label)
            break
        except ValueError:
            continue

    if background_sample is None:
        raise ValueError(f"Background sample not found in {filename}")

    # substract background
    sample.substract_background(background_sample)

    # read quench set
    all_quench.append(file_reader.quench_set)

    return sample


lsc_data_folder = "../../data/tritium_detection"
with open("../../data/general.json", "r") as f:
    general_data = json.load(f)

run_nb = general_data["general_data"]["run_nb"]


# read start time from general.json
all_start_times = []
for generator in general_data["generators"]:
    if generator["enabled"] is False:
        continue
    for irradiation_period in generator["periods"]:
        start_time = datetime.strptime(irradiation_period["start"], "%m/%d/%Y %H:%M")
        all_start_times.append(start_time)
start_time = min(all_start_times)


In [ ]:
stream = "IV"
sample_nb = "2"
vial_nb = "4"
filename = "REPEAT_IV-2-1_TO_2-10.csv"

sample = create_repeat_sample(
    label=f"1L-{stream}_{run_nb}-{sample_nb}-{vial_nb}",
    filename=f"{lsc_data_folder}/{filename}",
)

In [23]:
print(sample.name)
print(sample.counted_activities)
print(sample.avg_activity)
print(sample.stdev_activity)
print(sample.background_substracted)
print(sample.rep_number)

1L-IV_2-2-3
[5.234 5.224 5.207 5.177 5.352 5.228] becquerel
4.853999999999999 becquerel
0.06104096984812763 becquerel
True
6


In [ ]:
# create gas streams
gas_streams = {}
for stream, samples in general_data["tritium_detection"].items():
    stream_samples = []
    for sample_nb, sample_dict in samples.items():
        libra_samples = []
        if sample_dict["actual_sample_time"] is None:
            continue
        for vial_nb, filename in sample_dict["lsc_vials_filenames"].items():
            sample = create_sample(
                label=f"1L-{stream}_{run_nb}-{sample_nb}-{vial_nb}",
                filename=f"{lsc_data_folder}/{filename}",
            )
            libra_samples.append(sample)

        time_sample = datetime.strptime(
            sample_dict["actual_sample_time"], "%m/%d/%Y %H:%M"
        )
        stream_samples.append(LIBRASample(libra_samples, time=time_sample))
    gas_streams[stream] = GasStream(stream_samples, start_time=start_time)